In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd  /content/drive/MyDrive/NLP_project

/content/drive/MyDrive/NLP_project


In [ ]:
!pip install transformers -q
!pip install pandarallel -q
!pip install sentencepiece -q
!pip install neptune-client -q



In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
import neptune.new as neptune

from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset)

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
import itertools

In [ ]:
SEED=4
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device='cuda'

In [ ]:
dataset_path = "/content/drive/MyDrive/NLP_project/dataset"

path_predictions_folder = '/content/drive/MyDrive/NLP_project/quality'
save_model_folder = '/content/drive/MyDrive/NLP_project/quality'

In [ ]:
max_len_arg = 55
max_len_kp = 32
max_len_topic = 12
max_len_sent_1_sts = 58
max_len_sent_2_sts = 65
median_len_sent_1_sts = 9
median_len_sent_2_sts = 9
max_noun_encoded_feature_len = 66
max_dependency_encoded_feature_len = 66
max_len_tf_idf = 505
max_len_input = 128

model_with_no_token_types =['roberta', 'bart', 'albert','robertalarge']


In [ ]:
def make_dataset(tokenizer, args,topics, labels, max_len_input, model_with_no_token_types = model_with_no_token_types, model_name='roberta'):
    
    all_input_ids = []
    all_token_type_ids = []
    all_attention_masks = []
    all_labels = [] 
    
    # print("labels",labels,labels.shape)
    # print("topics",topics,topics.shape)
    # print("args",args,args.shape)
    # print("kps",kps,kps.shape)
    for arg,topic,label in zip(args,topics,labels) :

        arg = re.sub('[^a-zA-Z]', ' ', arg)
        # kp = re.sub('[^a-zA-Z]', ' ', kp)
        topic = re.sub('[^a-zA-Z]', ' ', topic)

        url = re.compile(r'https?://\S+|www\.\S+')
        arg = url.sub(r'',arg)
        # kp = url.sub(r'',kp)
        topic = url.sub(r'',topic)
        
        html=re.compile(r'<.*?>')
        arg = html.sub(r'',arg)
        # kp = html.sub(r'',kp)
        topic = html.sub(r'',topic)


        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        
        arg = emoji_pattern.sub(r'',arg)
        # kp = emoji_pattern.sub(r'',kp)
        topic = emoji_pattern.sub(r'',topic)

        if model_name in model_with_no_token_types:

          encoded_input = tokenizer(arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          #all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)

        else :

          encoded_input = tokenizer(arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          #all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)
          
    if model_name in model_with_no_token_types:
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels)
      
      dataset = TensorDataset(all_input_ids, all_attention_masks, all_labels)

    else :
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_token_type_ids = torch.tensor(all_token_type_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_labels = torch.tensor(all_labels) 

      dataset = TensorDataset(all_input_ids,all_token_type_ids, all_attention_masks, all_labels)

    return dataset

In [ ]:
df = pd.read_csv(dataset_path+'/arg_quality_rank_30k.csv')
num_total = df.shape[0]
print(num_total)


30497


In [ ]:
df_train = df.loc[df['set']=='train']
df_val = df.loc[df['set'] =='dev']
df_test  = df.loc[df['set']=='test']

print(df_train.shape,df_test.shape,df_val.shape)
# print(df_train.shape,df_val.shape,df_test.shape,df_sts.shape,df_arg30.shape)

df_train.head()

(20974, 7) (6315, 7) (3208, 7)


,argument,topic,set,WA,MACE-P,stance_WA,stance_WA_conf
0,"""marriage"" isn't keeping up with the times. a...",We should abandon marriage,train,0.846165,0.297659,1,1.000000
1,.a multi-party system would be too confusing a...,We should adopt a multi-party system,train,0.891271,0.726133,-1,1.000000
3,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,train,0.730395,0.225212,-1,1.000000
4,"100% agree, should they do that, it would be a...",We should abolish safe spaces,train,0.236686,0.004104,1,0.805517
8,A ban on naturopathy creates a cohesive front ...,We should ban naturopathy,train,0.753805,0.337724,1,1.000000


In [ ]:
model_name = 'robertalarge-trainquality'
model_path = '/content/drive/MyDrive/Colab Notebooks/90000_datapoints_pretrained_model'
tokenizer_model_path = 'roberta-large'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_path)

In [ ]:
train_dataset = make_dataset(tokenizer, df_train['argument'], df_train['topic'],df_train['MACE-P'], max_len_input, model_with_no_token_types,model_name)

In [ ]:
val_dataset = make_dataset(tokenizer, df_val['argument'], df_val['topic'],df_val['MACE-P'], max_len_input, model_with_no_token_types,model_name)  

In [ ]:
class NonPoolerTransformer(nn.Module):

    def __init__(self):
        super(NonPoolerTransformer, self).__init__()
        
        #Instantiating Pre trained model object 
        self.model_layer = AutoModel.from_pretrained(model_path)

        #Layers
        # the first dense layer will have 768 if base model is used and 
        # 1024 if large model is used

        self.dense_layer_1 = nn.Linear(1024, 256)
        self.dropout = nn.Dropout(0.4)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.2)
        self.cls_layer = nn.Linear(128, 1, bias = True)
        self.sigmoid = nn.Sigmoid()#for MACE-P

    def forward(self,input_ids, attention_masks):

        hidden_state = self.model_layer(input_ids=input_ids, attention_mask=attention_masks)[0]
        pooled_output = hidden_state[:, 0]

        x = self.dense_layer_1(pooled_output)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)
        
        logits  = self.cls_layer(x_2)
        output  = self.sigmoid(logits)

        return output

In [ ]:
model = NonPoolerTransformer()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = model.to(device)

In [ ]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 5
ACCUMULATION_STEPS = 2
DROPOUT = 0.2


In [ ]:
PARAMS = {'model_name': model_name,'model_path': model_path,'lr': LEARNING_RATE, 'epoch_nr': EPOCHS, 'batch_size': BATCH_SIZE, 'accumulation_steps': ACCUMULATION_STEPS,'dropout': DROPOUT}

In [ ]:
def evaluate_model(test_dataset,df, model,  model_name, mode = 'train'):
    
    save_predictions_name = model_name+ '__VAL_PREDS_'+ 'SEED_'+ str(SEED) + '_dense_layer' +'_epoc_'+ str(EPOCHS)+'_lr_'+ str(LEARNING_RATE)+'_b_s_'+ str(BATCH_SIZE) +'_accumulation_steps_'+ str(ACCUMULATION_STEPS) +'_input_type_Quality_MACE-P'

    y_preds = []
    val_losses = []
    criterion = nn.BCELoss()
    list_of_batch_losses = []
    
    if mode in ['train','val']:
        test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    with torch.no_grad():
        acc_epoch = []
        
        epoch_iterator = tqdm(test_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.eval()
            
            if model_name in model_with_no_token_types:
                b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
                ypred = model(b_input_ids, b_input_mask)
            else:
                b_input_ids,b_token_type, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
                ypred = model(b_input_ids, b_input_mask,b_token_type)
                
            b_labels_copy = torch.reshape(b_labels, (b_labels.shape[0], 1))
            loss_batch = criterion(ypred, b_labels_copy.float())
            list_of_batch_losses.append(loss_batch.detach().cpu().numpy())
            
            ypred = ypred.cpu().numpy()
            b_labels = batch[-1].cpu().detach().numpy()
        
            ypred = np.hstack(ypred)
            y_preds.append(ypred)
    
    epoch_loss = np.mean(list_of_batch_losses)
    val_losses.append(epoch_loss)
    
    
    args = df['argument']
    # kps = df['key_point_id']
    true_labels = df['MACE-P']
    topics = df['topic']
    #stances = df['stance']
    all_preds = []

    for i in tqdm(range(len(y_preds))):
      for p in y_preds[i]:
        all_preds.append(p)
            
    print('Val evaluation....')
    
    pred_file = pd.DataFrame({"argument" : args, "topic": topics, "MACE-P": all_preds})
    args = {}
    kps = {}

    for arg,topic,score in zip(pred_file['argument'],pred_file['topic'],pred_file['MACE-P']):
        args[arg] = {}

    for arg,topic,score in zip(pred_file['argument'],pred_file['topic'],pred_file['MACE-P']):
        args[arg][topic] = score

    with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
        fp.write(json.dumps(args))
        fp.close()
    
    # arg_df, kp_df, labels_df = load_kpm_data(path_dataset, subset="dev")
    # merged_df = get_predictions(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', labels_df, arg_df, kp_df)
    
    # evaluate_predictions(merged_df,name = 'val')

    return all_preds,true_labels, val_losses

In [ ]:
 def train_and_evaluate(train_dataset,df,model, filepath, model_name, batch_size = BATCH_SIZE, learning_rate = LEARNING_RATE, epochs = EPOCHS,accumulation_steps = ACCUMULATION_STEPS):
  
  train_losses = []
  val_losses = []
    
  save_model = model_name+ '_SEED_'+ str(SEED) +'_dense_layer' +'_epoc_'+ str(epochs)+'_lr_'+ str(learning_rate)+'_b_s_'+ str(batch_size ) +'_accumulation_steps_'+ str(accumulation_steps) +'_input_type_Quality_MACE-P' 
  save_predictions_name  = model_name+ '__TRAIN_PREDS_'+ 'SEED_'+ str(SEED) + '_dense_layer' +'_epoc_'+ str(epochs)+'_lr_'+ str(learning_rate)+'_b_s_'+ str(batch_size ) +'_accumulation_steps_'+ str(accumulation_steps) +'_input_type_Quality_MACE-P'

  training_dataloader = DataLoader(train_dataset, batch_size )
  total_steps = len(training_dataloader) * epochs
  no_decay = ['bias', 'LayerNorm.weight']
  
  optimizer_grouped_parameters = [
                                  {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                  {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
                                  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

  criterion = nn.BCELoss()
    
  model.zero_grad()

  for epoch_i in tqdm(range(epochs)):
    y_preds = []
    y_val = []
    list_of_batch_losses = []
    epoch_iterator = tqdm(training_dataloader, desc="Iteration")
    model.train()
    
    for step, batch in enumerate(epoch_iterator):
      if model_name in model_with_no_token_types:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        outputs = model(b_input_ids, b_input_mask)
      else:
        b_input_ids,b_token_type, b_input_mask,b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        outputs = model(b_input_ids, b_input_mask,b_token_type)
            
      b_labels = torch.reshape(b_labels, (b_labels.shape[0], 1))
      loss = criterion(outputs, b_labels.float())
             
      list_of_batch_losses.append(loss.detach().cpu().numpy())
      
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
      ypred = outputs.detach().cpu().numpy()
      b_labels = batch[-1].cpu().detach().numpy()
      ypred = np.hstack(ypred)
      y_preds.append(ypred)

      if (step+1) % accumulation_steps == 0:
        optimizer.step()
        scheduler.step()
        model.zero_grad()


    epoch_loss = np.mean(list_of_batch_losses)
    train_losses.append(epoch_loss)
    
    args = df['argument']
    # kps = df['key_point']
    true_labels = df['MACE-P']
    topics = df['topic']
    # stances = df['stance']
    all_preds = []
    
    for i in tqdm(range(len(y_preds))):
      for p in y_preds[i]:
        all_preds.append(p)

    print('Train evaluation....')
    
    pred_file = pd.DataFrame({"argument" : args, "topic": topics, "MACE-P": all_preds})
    args = {}
    kps = {}

    for arg,kp,score in zip(pred_file['argument'],pred_file['topic'],pred_file['MACE-P']):
        args[arg] = {}

    for arg,kp,score in zip(pred_file['argument'],pred_file['topic'],pred_file['MACE-P']):
        args[arg][kp] = score

    with open(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', 'w') as fp:
        fp.write(json.dumps(args))
        fp.close()
    
    #arg_df, kp_df, labels_df = load_kpm_data(dataset_path, subset="train")#
    #merged_df = get_predictions(path_predictions_folder + save_predictions_name + '_' + 'predictions.p.', labels_df, arg_df, kp_df)#
    
    #evaluate_predictions(merged_df,name = 'train')
    
    _,_, val_epoch_loss = evaluate_model(val_dataset,df_val, model, model_name, mode = 'val')#
    val_losses.append(val_epoch_loss)
    

  torch.save(model, save_model_folder +save_model+'.pt')

  print("Model is saved as : ",save_model)
  print("Use this to load the model")
    
  return save_model,train_losses, val_losses

In [ ]:
save_model, train_losses, val_losses = train_and_evaluate(train_dataset,df_train, model, save_model_folder, model_name = 'robertalarge')

  0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/656 [00:00<?, ?it/s]

  0%|          | 0/656 [00:00<?, ?it/s]

Train evaluation....


Iteration:   0%|          | 0/101 [00:00<?, ?it/s]

...

  0%|          | 0/656 [00:00<?, ?it/s]

Train evaluation....


Iteration:   0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Val evaluation....
Model is saved as :  robertalarge_SEED_4_dense_layer_epoc_5_lr_2e-05_b_s_32_accumulation_steps_2_input_type_Quality_MACE-P
Use this to load the model


In [ ]:
print(train_losses)

In [ ]:
print(val_losses)

In [ ]:
print(save_model)

In [ ]:
def load_predictions(predictions_dir):
    arg =[]
    kp = []
    scores = []
    invalid_keypoints = set()
    with open(predictions_dir, "r") as f_in:
        res = json.load(f_in)
        for arg_id, kps in res.items():
          print(arg_id,kps)

In [ ]:
load_predictions('/content/drive/MyDrive/NLP_project/qualityrobertalarge__VAL_PREDS_SEED_4_dense_layer_epoc_5_lr_2e-05_b_s_32_accumulation_steps_2_input_type_Quality_MACE-P_predictions.p.')

In [ ]:
load_predictions('/content/drive/MyDrive/NLP_project/qualityrobertalarge__TRAIN_PREDS_SEED_4_dense_layer_epoc_5_lr_2e-05_b_s_32_accumulation_steps_2_input_type_Quality_MACE-P_predictions.p.')